Inspired by https://f1metrics.wordpress.com/2014/10/03/building-a-race-simulator/



In [1]:
import fastf1 as ff1
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
from race import *
from constants import *
from practice import *
from scipy import stats

# Enable the cache by providing the name of the cache folder
ff1.Cache.enable_cache('cache')

#### FP2

In [2]:
FP2 = ff1.get_session(2022, 'Hungary', 'FP2')
FP2.load();

core           INFO 	Loading data for Hungarian Grand Prix - Practice 2 [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '55', '47', '44', '31', '24', '23', '22', '20', '18', '16', '14', '11', '10', '6', '5', '4', '3', '63', '77']


In [3]:
# plot_dry_tyre_models_all_drivers(FP2.laps, get_all_driver_names(FP2.laps))

#### Data Wrangling - Getting the Race Pace of each driver from FP2 Long Run Simulation

In [66]:
# Create a new dataframe for storing drivers' racepace
drivers_race_pace = pd.DataFrame(columns = ['Driver', 'MeanLapTime', 'Compound', 'NoOfLaps', 'LapTimes'])
drivers_race_pace

,Driver,MeanLapTime,Compound,NoOfLaps,LapTimes


In [67]:
for driver in get_all_driver_names(FP2.laps):
    long_pace_tuple = extract_long_run_pace_from_longest_practice_stint(FP2.laps, driver)
    
    # Create a new entry to the racepace dataframe by creating a row, which will be appended to the dataframe (drivers_race_pace) in the next line.
    # Note the order of the items in the list.
    new_row = [driver,  sum(long_pace_tuple[0]) /  len(long_pace_tuple[0]), long_pace_tuple[1], len(long_pace_tuple[0]), long_pace_tuple[0]]
    # Append the new entry.
    drivers_race_pace.loc[len(drivers_race_pace)] = new_row
    
drivers_race_pace = drivers_race_pace.sort_values('MeanLapTime')
drivers_race_pace.sort_index()

# TODO: For the drivers with NoOfLaps less than 3, we can "interpolate?extrapolate?guess?" their laptime by getting the RANGE of the laptime
# of the remaining drivers and assign a laptime based on their TEAM, with teams with faster cars getting a better laptime. For example, if the team
# is the 6th best team, an example of the guesswork value can be:
# laptime = min + (max - min) * 6 / NO_OF_TEAMS
# Note that this is assuming that the time delta between each team is constant. This is a simple solution for now, but not necessarily fair.

# TODO: Consider the different tyre compounds used

,Driver,MeanLapTime,Compound,NoOfLaps,LapTimes
0,VER,84.173000,MEDIUM,8,"[83.771, 83.941, 84.012, 84.18100000000001, 84..."
1,SAI,83.624000,MEDIUM,7,"[83.185, 83.22500000000001, 83.44000000000001,..."
2,MSC,86.678000,MEDIUM,6,"[86.072, 86.894, 87.257, 86.54100000000001, 86..."
3,HAM,80.853000,MEDIUM,1,[80.85300000000001]
4,OCO,84.055000,SOFT,1,[84.055]
5,ZHO,85.335375,MEDIUM,8,"[84.90700000000001, 84.49300000000001, 84.748,..."
6,ALB,85.978000,HARD,7,"[86.18700000000001, 85.54400000000001, 85.43, ..."
7,TSU,85.604250,MEDIUM,12,"[84.89200000000001, 84.781, 85.209, 85.211, 85..."
8,MAG,85.827600,MEDIUM,10,"[85.35900000000001, 85.78800000000001, 85.683,..."
9,STR,84.797700,SOFT,10,"[83.80900000000001, 84.14200000000001, 84.397,..."


In [68]:
for driver in get_all_driver_names(FP2.laps):
    df = FP2.laps.pick_driver(driver).pick_accurate()
    drivers_race_pace.loc[drivers_race_pace['Driver'] == driver, 'SpeedFL'] = max(df['SpeedFL'])
    drivers_race_pace.loc[drivers_race_pace['Driver'] == driver, 'SpeedST'] = max(df['SpeedST'])

drivers_race_pace

,Driver,MeanLapTime,Compound,NoOfLaps,LapTimes,SpeedFL,SpeedST
3,HAM,80.853000,MEDIUM,1,[80.85300000000001],253.0,302.0
1,SAI,83.624000,MEDIUM,7,"[83.185, 83.22500000000001, 83.44000000000001,...",257.0,NaN
10,LEC,83.698286,MEDIUM,7,"[83.57400000000001, 83.552, 83.435, 83.4, 83.7...",253.0,303.0
19,BOT,83.834000,MEDIUM,1,[83.834],255.0,305.0
4,OCO,84.055000,SOFT,1,[84.055],253.0,304.0
0,VER,84.173000,MEDIUM,8,"[83.771, 83.941, 84.012, 84.18100000000001, 84...",255.0,305.0
17,RIC,84.487800,SOFT,5,"[84.21600000000001, 83.796, 84.15400000000001,...",255.0,304.0
15,VET,84.585000,MEDIUM,8,"[84.03500000000001, 84.138, 84.71000000000001,...",255.0,305.0
12,PER,84.781500,MEDIUM,10,"[84.558, 84.578, 84.39, 84.577, 84.727, 84.972...",254.0,306.0
9,STR,84.797700,SOFT,10,"[83.80900000000001, 84.14200000000001, 84.397,...",256.0,308.0
